<a href="https://colab.research.google.com/github/GOE84/Gold-Price-Prediction/blob/main/Gold_Price_Prediction_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gold Price Prediction

**# นำเข้าไลบรารีที่จำเป็นสำหรับการวิเคราะห์ข้อมูล**


*   `import yfinance as yf` : ดึงข้อมูลทางการเงินจาก Yahoo Finance
*   `import pandas as pd` : สำหรับการจัดการและวิเคราะห์ข้อมูลในรูปแบบตาราง `(DataFrame)`
*   `import matplotlib.pyplot as plt` : นำเข้าโมดูล pyplot จากไลบรารี matplotlib ซึ่งใช้สำหรับสร้างกราฟและภาพข้อมูล โดยตั้งชื่อย่อว่า `plt`


In [ ]:
import sys
!{sys.executable} -m pip install yfinance

In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

# Retrieve gold price data.


---



`yfinance` เพื่อดาวน์โหลดข้อมูล`ราคาทองคำ` โดยใช้สัญลักษณ์ `'GC=F'` ซึ่งเป็นสัญลักษณ์สำหรับทองคำ และกำหนดช่วงเวลาเริ่มต้นที่ 1 มกราคม 2010 และสิ้นสุดที่ 31 ธันวาคม 2024 ค่ะ ข้อมูลที่ดาวน์โหลดมาจะถูกเก็บไว้ในตัวแปรชื่อ `gold_data`\

In [ ]:
gold_data = yf.download("GC=F", start="2010-01-01", end="2024-12-31")

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(gold_data.index, gold_data['Close'], label='Gold Close Price')
plt.title('Gold Price (GC=F) Over Time')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
print(gold_data.head())



> เซลล์นี้จะแสดงข้อมูล 5 แถวแรกของ DataFrame ที่ชื่อว่า gold_data



In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 8))


axes[0].plot(gold_data.index, gold_data['Volume'], label='Gold Trading Volume', color='orange')
axes[0].set_title('Gold Trading Volume (GC=F) Over Time')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Volume')
axes[0].legend()
axes[0].grid(True)

gold_monthly = gold_data['Close'].resample('ME').mean()

axes[1].plot(gold_monthly, label='Monthly Average Price', color='blue') # Changed color for distinction
axes[1].set_title('Monthly Average Gold Price')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Price (USD)')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

# Model format

**ARIMA (AutoRegressive Integrated Moving Average)**
> (Traditional): ใช้โมเดล ARIMA

**โมเดลนี้ประกอบด้วย 3 ส่วนหลักที่เราต้องหาค่าที่ดีที่สุด (p, d, q):**
*   p (AR): การเอาค่าในอดีตมาทำนายอนาคต
*   d (I): การทำให้ข้อมูล "นิ่ง" (Stationary) โดยการลบค่าปัจจุบันด้วยค่าก่อนหน้า
*   q (MA): การเอาความผิดพลาด (Error) ในอดีตมาช่วยปรับจูน



พยากรณ์ข้อมูลอนุกรมเวลา `(Time Series)`









In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller

In [ ]:
# 1. เช็คความนิ่งของข้อมูล (Stationarity Check)
def check_stationarity(series):
    result = adfuller(series)
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    if result[1] <= 0.05:
        print("ข้อมูลนิ่งแล้ว (Stationary)")
    else:
        print("ข้อมูลยังไม่นิ่ง (Non-Stationary) ต้องทำการ Diff")

check_stationarity(gold_monthly.dropna())

**สร้างและฝึกสอนโมเดล ARIMA**

> หมายเหตุ: (5,1,0) เป็นค่าเริ่มต้นทั่วไป p=5, d=1, q=0



In [ ]:
model = ARIMA(gold_monthly, order=(5, 1, 0))
model_fit = model.fit()

In [ ]:

forecast_steps = 12
forecast = model_fit.get_forecast(steps=forecast_steps)
forecast_mean = forecast.predicted_mean
conf_int = forecast.conf_int()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(gold_monthly, label='Historical Price')
plt.plot(forecast_mean, color='red', label='Forecast (Next 12 Months)')
plt.fill_between(conf_int.index, conf_int.iloc[:, 0], conf_int.iloc[:, 1], color='pink', alpha=0.3)
plt.title('Gold Price Prediction with ARIMA')
plt.legend()
plt.show()

# "ค่าที่ดีที่สุด" (Optimal Parameters)



In [ ]:
!pip install pmdarima